In [26]:
codigo_teste = 'inteiro x;texto y; x recebe 3; enquanto(x for_menor_que 6) se (x for_menor_que 3) x recebe 1; caso_contrario x recebe 2;'

In [27]:
#imports
from rply import LexerGenerator
from rply.token import BaseBox

In [28]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMERO', r'\d+')
lg.add('MAIS', r'mais')
lg.add('MENOS', r'menos')
lg.add('MULTIPLICACAO', r'multiplicado_por')
lg.add('DIVISAO', r'dividido_por')
lg.add('ABRE_PARENTESES', r'\(')
lg.add('FECHA_PARENTESES', r'\)')

lg.add('INTEIRO', r'inteiro')
lg.add('TEXTO', r'texto')
lg.add('SE', r'se')
lg.add('CASO_CONTRARIO', r'caso_contrario')
lg.add('ENQUANTO', r'enquanto')

lg.add('COMPARADOR','for_igual_a')
lg.add('COMPARADOR','for_diferente_de')
lg.add('COMPARADOR','for_maior_ou_igual_a')
lg.add('COMPARADOR','for_menor_ou_igual_a')
lg.add('COMPARADOR','for_maior_que')
lg.add('COMPARADOR','for_menor_que')

lg.add('RECEBE', r'recebe')
lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('PONTO_VIRGULA', r';')

lg.ignore('\s+')

lexer = lg.build()

In [29]:
tokens = lexer.lex("x gustavo y inteiro x recebe 10 y recebe 20")
for token in tokens:
    print(token)

Token('ID', 'x')
Token('ID', 'gustavo')
Token('ID', 'y')
Token('INTEIRO', 'inteiro')
Token('ID', 'x')
Token('RECEBE', 'recebe')
Token('NUMERO', '10')
Token('ID', 'y')
Token('RECEBE', 'recebe')
Token('NUMERO', '20')


In [31]:
#ÁRVORE SINTÁTICA PREPARADA PARA RECEBER VISITORS

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,stmts):
        self.decls = decls
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp):
        self.id = id
        self.tp = tp


    def accept(self, visitor):
        visitor.visit_vardecl(self)

class Statements(BaseBox):
    def __init__(self, stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)


class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_while(self)

class For(BaseBox):
    def __init__(self, idinic, exprinic, expr1, comp, expr2, idincr, exprincr, ie1):
        self.idinic=idinic
        self.exprinic=exprinic
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.idincr=idincr
        self.exprincr=exprincr
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_for(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass


class Sub(BinaryOp):
  pass


class Mul(BinaryOp):
  pass


class Div(BinaryOp):
  pass


In [32]:
#ANALISADOR SINTÁTICO

from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMERO', 'ABRE_PARENTESES', 'FECHA_PARENTESES',
     'MAIS', 'MENOS', 'MULTIPLICACAO', 'DIVISAO', 'INTEIRO', 'TEXTO', 'ID','PONTO_VIRGULA',
     'RECEBE','COMPARADOR','SE','CASO_CONTRARIO','ENQUANTO'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['MAIS', 'MENOS']),
        ('left', ['MULTIPLICACAO', 'DIVISAO'])
    ]
)

@pg.production('prog : vardecls statements')
def prog(p):
    return Prog(p[0],p[1])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################

@pg.production('vardecls : vardecl')
def vardecls(p):
    return VarDecls(p[0],None)

@pg.production('vardecls : vardecl vardecls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('vardecl : TEXTO ID PONTO_VIRGULA')
def vardecl_string(p):
    return VarDecl(p[1].getstr(), "texto")

@pg.production('vardecl : INTEIRO ID PONTO_VIRGULA')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "inteiro")

##################################################
# COMANDOS - CASO ABERTO
##################################################

@pg.production('statements : openstatement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : openstatement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('openstatement : ID RECEBE expression PONTO_VIRGULA')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('openstatement : SE ABRE_PARENTESES expression COMPARADOR expression FECHA_PARENTESES openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],None)


@pg.production('openstatement : SE ABRE_PARENTESES expression COMPARADOR expression FECHA_PARENTESES closedstatement CASO_CONTRARIO openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('openstatement : ENQUANTO ABRE_PARENTESES expression COMPARADOR expression FECHA_PARENTESES openstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])


##################################################
# COMANDOS - CASO FECHADO
##################################################

@pg.production('closedstatement : ID RECEBE expression PONTO_VIRGULA')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('closedstatement : SE ABRE_PARENTESES expression COMPARADOR expression FECHA_PARENTESES closedstatement CASO_CONTRARIO closedstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('closedstatement : ENQUANTO ABRE_PARENTESES expression COMPARADOR expression FECHA_PARENTESES closedstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

@pg.production('expression : NUMERO')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : ABRE_PARENTESES expression FECHA_PARENTESES')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression MAIS expression')
@pg.production('expression : expression MENOS expression')
@pg.production('expression : expression MULTIPLICACAO expression')
@pg.production('expression : expression DIVISAO expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'MAIS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MENOS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MULTIPLICACAO':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIVISAO':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [33]:
# VISITOR PARA TABELA DE SÍMBOLOS

ST={}

class Visitor(object):
  pass

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)

    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_vardecl(self, d):
        ST[d.id]=d.tp


In [34]:
arvore=parser.parse(lexer.lex(codigo_teste))
arvore.accept(SymbolTable())
print(ST)

{'x': 'inteiro', 'y': 'texto'}


In [35]:
#VISITOR PARA DECORAR TIPOS

class Decorator(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        if i.id in ST:
          i.decor_type=ST[i.id]
        else:
          raise AssertionError('id não declarado')
        i.expr.accept(self)

    def visit_ifelse(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
        if i.ie2!=None:
          i.ie2.accept(self)

    def visit_while(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)

    def visit_for(self, i):
        if i.idinic in ST:
          i.inicdecor_type=ST[i.idinic]
        else:
          raise AssertionError('id não declarado')
        i.exprinic.accept(self)
        i.expr1.accept(self)
        i.expr2.accept(self)
        if i.idincr in ST:
          i.incrdecor_type=ST[i.idincr]
        else:
          raise AssertionError('id não declarado')
        i.exprincr.accept(self)
        i.ie1.accept(self)


    def visit_id(self, i):
        if i.value in ST:
          i.decor_type=ST[i.value]
        else:
          raise AssertionError('id não declarado')


    def visit_number(self, i):
        i.decor_type="inteiro"


    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="inteiro" and a.right.decor_type=="inteiro":
          a.decor_type="inteiro"


    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="inteiro" and a.right.decor_type=="inteiro":
          a.decor_type="inteiro"

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="inteiro" and a.right.decor_type=="inteiro":
          a.decor_type="inteiro"

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="inteiro" and a.right.decor_type=="inteiro":
          a.decor_type="inteiro"



In [37]:
arvore=parser.parse(lexer.lex(codigo_teste))
arvore.accept(SymbolTable())
arvore.accept(Decorator())

In [38]:
# VISITOR - TYPE VERIFIER

class TypeVerifier(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        if i.decor_type!=i.expr.decor_type:
          raise AssertionError('Erro de tipo')

    def visit_ifelse(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('Erro de tipo')

    def visit_while(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('Erro de tipo')

    def visit_for(self, i):
        # verificação da inicialização do for
        if i.inicdecor_type!=i.exprinic.decor_type:
          raise AssertionError('Erro de tipo')

        # verificação da comaração do for
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('Erro de tipo')

        # verificação da atualização do for
        if i.incrdecor_type!=i.exprincr.decor_type:
          raise AssertionError('Erro de tipo')


In [39]:
arvore=parser.parse(lexer.lex(codigo_teste))
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())